In [ ]:
# NOTE: requires the *.zip archives in 1_data/3_eo_rapid/ to be unzipped

In [ ]:
from hydromt_sfincs import SfincsModel
import hydromt
from os.path import join
import xarray as xr
import numpy as np
import glob
import os
import sys

In [ ]:
# requires the model to be setup in step 2_experiment/21_setup_sfincs_models.ipynb
root = r'../../3_models/SFINCS/00_base_100m'
mod0 = SfincsModel(root, mode='r')

In [ ]:
# read and resample images to model domain
ddir = r'../../1_data/3_eo_rapid'
dates = ['20190319', '20190320', '20210125']
types = ['flooding', 'Dry']

rm_fns = []
for date in dates:
    for name in types:
        fns = glob.glob(join(ddir, date, f'{name}*.tif'))
        da_lst = []
        for fn in fns:
            print(fn)
            da_obs0 = hydromt.open_raster(fn).load().astype(np.int8)
            da_obs0.raster.set_nodata(-1)
            try:
                da_obs0 = da_obs0.raster.reproject_like(mod0.staticmaps, method='max')
                da_lst.append(da_obs0)
            except IndexError:
                da_obs0.close()
                rm_fns.append(fn)
                print('out of bounds')
                pass
            
        if len(da_lst) > 0:
            print(f'concatenate {len(da_lst)} files')
            da = xr.concat(da_lst, dim='img').max('img').load().astype(np.uint8)
            da = da.where(da==1,0)
            da.raster.set_nodata(0)
            da.raster.to_raster(join(root, 'gis', f'{name.lower()}_{date}.tif'), compress='deflate')

if len(rm_fns) > 0:
    print(f'deleting {len(rm_fns)} files')
    for fn in rm_fns:
        os.unlink(fn)